In [1]:
%env METIS_DLL=/home/juan/Descargas/metis-5.0/build/Linux-x86_64/libmetis/libmetis.so

env: METIS_DLL=/home/juan/Descargas/metis-5.0/build/Linux-x86_64/libmetis/libmetis.so


In [14]:
import scipy

scipy.version.version

'1.5.2'

In [15]:
from scipy.sparse import lil_matrix, csr_matrix
import numpy as np
#import metis
from clustering_evaluation import computeEvaluationMeasures, MEASURES
import glob
from tabulate import tabulate
from tqdm import tqdm
import os

def _loadSparseCSRFromDisk(fullPath):
    spD = None
    with open(fullPath, 'rb') as f:
        data = np.load(f)
        indices = np.load(f)
        indptr = np.load(f)
        spD = csr_matrix((data, indices, indptr))
        # convert this nxk matrix into a similarity.
    return spD

def _SaveSparseCSRToDisk(denseFn, spM):
    """
    denseFn: output file 
    spM: sparse matrix to convert and store.
    """
    csrOut = "{0}.npy".format(denseFn[: -(len(denseFn.split('.')[-1])+1)]) # replaces the file extension.
    print(csrOut)
    #M  = np.loadtxt(denseFn, delimiter=',')
    #spM = csr_matrix(M)
    with open(csrOut, 'wb') as f:
        np.save(f, spM.data)
        np.save(f, spM.indices)
        np.save(f, spM.indptr)
    """
    When loading
    >> with open('test.npy', 'rb') as f:
    >>     a = np.load(f)
    >>     b = np.load(f)
    >> csr_matrix((data, indices, indptr))
    """

def getLabelsFromFile(fullPath):
    v = np.loadtxt(fullPath, delimiter=',', dtype=np.int)
    return v

def detectPostfixesInDir(fullPath, dirName):
    fnLst = glob.glob('{0}/{1}/*labels_*.out'.format(fullPath, dirName))
    # labels_NCF_DS_
    postFixes = [f.split(os.sep)[-1].replace('labels_NCF_DS_','').replace('.out','') for f in fnLst]
    return postFixes

#path = '/home/juan/Insync/juan.zamora@pucv.cl/Google Drive/Research - Multiview and Collaborative Clustering/code/handwritten_tezt/similarity_matrices_after_merging'

In [16]:
from scipy.sparse import csc_matrix,csr_matrix, lil_matrix
from itertools import product
#from spectral_embedding import *
from sklearn.cluster import MiniBatchKMeans
from sklearn.manifold import SpectralEmbedding, spectral_embedding
from clustering_evaluation import computeEvaluationMeasures, MEASURES
from sklearn.metrics.pairwise import pairwise_kernels


def getSimClus(D):
    """
    Computes a matrix that contains the intersection of items
    D is a csc_matrix with n rows denoting data points and k columns denoting clusters.
    D_ij denotes the membership score of data point i in cluster j.
    """
    D_asCol = csc_matrix(D) # more efficient for making column slices

    # create a kxk matrix with zeros
    k = D.shape[1]
    simClus = np.zeros((k,k))
    # compare each pair of clusters from D
    for i in range(k):
        sClust_i = set(D_asCol[:,i].nonzero()[0])
        for j in range(i+1,k):
            sClust_j = set(D_asCol[:,j].nonzero()[0])
            simClus[i,j] = len(sClust_i & sClust_j) / len(sClust_i | sClust_j)
            simClus[j,i] = simClus[i,j]
    return simClus


def compute_aSRS(D, DC=1.0):
    # aim: compute the approximate SimRankSimilarity matrix (aSRS).
    # compute for each pair of data points i,j the total sum of simclus values 
    n = D.shape[0]
    
    simClus = getSimClus(D)

    aSRS = np.zeros((n,n))
    for i in range(n):
        nbors_i = D[i].nonzero()[1]
        n_nbors_i = len(nbors_i)
        for j in range(i+1, n):
            nbors_j = D[j].nonzero()[1]
            n_nbors_j = len(nbors_j)
            aSRS[i,j] = (DC/(n_nbors_i*n_nbors_j))*np.sum([simClus[ca,cb] for ca, cb in list(product(nbors_i, nbors_j))])
            aSRS[j,i] = aSRS[i,j]
    return aSRS

# old method, use it only as an example. Below it is the correct version!
def _deprecated_perform_evaluate_runs(dsDirPath, nruns=10, threshold=0.0, embDim=4, kernelFn='rbf'):
    trueLblsFname = "{0}/{1}/{2}".format(path, dsDirPath, 'labels.true')
    trueL = getLabelsFromFile(trueLblsFname)
    postFixesDS = detectPostfixesInDir(path, dsDirPath)

    dsresults = []
    header = ['seed']
    header.extend(MEASURES)
    np.random.seed(87009)
    n_clusters = np.unique(trueL).shape[0]
    runSeed = np.random.randint(0, high=2e8)
    for runPostfix in tqdm(postFixesDS):#tqdm(postFixesDS):
        #print("{0}/{1}{2}_csr.npy".format(dsDirPath, 'simmatrix_DS_', runPostfix))        
        simFname = "{0}/{1}/{2}{3}_csr.npy".format(path, dsDirPath, 'simmatrix_DS_', runPostfix)
        D = _loadSparseCSRFromDisk(simFname)

        #aSRS = compute_aSRS(D)
        #outSRSFn = "{0}/{1}/{2}{3}.npy".format(path, dsDirPath, 'aSRS_DS_', runPostfix)
        #np.save(outSRSFn, aSRS)
        ##aSRS = np.load(outSRSFn)
        
        # builds the embedded vectors
        embedding = spectral_embedding(pairwise_kernels(D, metric=kernelFn), n_components=embDim, norm_laplacian=True, drop_first=True)
        
        #spEmbMod = SpectralEmbedding(n_components=20, affinity='nearest_neighbors', n_jobs=4, eigen_solver='arpack', random_state=runSeed)        
        #embedding = spEmbMod.fit_transform(D.toarray())
        
        #outEmbFn = "{0}/{1}/{2}{3}.npy".format(path, dsDirPath, 'embedding_DS_', runPostfix)
        #np.save(outEmbFn, embedding)
        #embedding = np.load(outEmbFn)
        
        # old, not working!
        #spectralEmb = SpectralEmbedding(embedding_dimension = 30, node_weights = 'degree')
        #spectralEmb.fit(aSRS)
        #embedding = spectralEmb.embedding_
        
        runDsResults = [] # a single run passes through all the npy matrices with the same seed        
        for run in tqdm(range(nruns)): # the partitioning method has a stochastic part, hence several runs are needed.
            kmeans = MiniBatchKMeans(n_clusters, random_state=np.random.randint(0, high=2e8))
            kmeans.fit(embedding)
            pred_labels = list(kmeans.labels_)

            perf = computeEvaluationMeasures(trueL, pred_labels)
            runDsResults.append([perf[m] for m in MEASURES])

        runRow = [runSeed]
        runRow.extend(list(np.mean(runDsResults, axis=0)))
        dsresults.append( runRow ) # average

    dsresults.append( ['Average'] + list(np.mean(dsresults, axis=0)[1:]) )
    print(tabulate(dsresults, headers=header, tablefmt='github', floatfmt='.6f', showindex=True))    

In [5]:
def embed_and_consensus(dsDirPath, nruns=10, threshold=0.0, embDim=4, kernelFn='rbf', precomputed_sim=False):
    """
    precomputed_sim: Signals if the file with the soft-clustering based similarity matrix is already computed and stored.
    """
    trueLblsFname = "{0}/{1}/{2}".format(path, dsDirPath, 'labels.true')
    trueL = getLabelsFromFile(trueLblsFname)
    postFixesDS = detectPostfixesInDir(path, dsDirPath)

    dsresults = []
    header = ['seed']
    header.extend(MEASURES)
    np.random.seed(87009)
    n_clusters = np.unique(trueL).shape[0]
    runSeed = np.random.randint(0, high=2e8)
    
    t = tqdm(postFixesDS, leave=False)
    
    for runPostfix in t:

        simFname = "{0}/{1}/{2}{3}_csr.npy".format(path, dsDirPath, 'dotmatrix_DS_', runPostfix)
        
        if not precomputed_sim:
            softClusFN = "{0}/{1}/{2}{3}.out".format(path, dsDirPath, 'simmatrix_DS_', runPostfix)
            print("Opening soft-clustering matrix from {0}".format(softClusFN))
            scM = csr_matrix(np.loadtxt(softClusFN, delimiter=',') )# soft-clustering matrix        
                        
            # similarity computation (numpy pairwise... for dense matrices and .dot for sparse.)
            #simM = pairwise_kernels(scM, metric=kernelFn) # dot prod between soft-clus matrix and itself (transp.)
            simM = scM.dot(scM.T)
            
            print("Storing similarity matrix in sparse format to {0}".format(simFname))
            _SaveSparseCSRToDisk("{0}/{1}/{2}{3}_csr.npy".format(path, dsDirPath, 'dotmatrix_DS_', runPostfix), simM)
        else:
            t.set_description("Opening soft-clustering based similarity matrix from {0}".format(simFname))
            #print("Opening soft-clustering based similarity matrix from {0}".format(simFname))
            simM = _loadSparseCSRFromDisk(simFname)
        
        embedding = spectral_embedding(simM, n_components=embDim, norm_laplacian=True, drop_first=True)        
        
        runDsResults = [] # a single run passes through all the npy matrices with the same seed        
        t2 = tqdm(range(nruns), leave=False)
        for run in t2: # the partitioning method has a stochastic part, hence several runs are needed.
            t2.set_description("run {0}/{1}".format(run, nruns))
            kmeans = MiniBatchKMeans(n_clusters, random_state=np.random.randint(0, high=2e8))
            kmeans.fit(embedding)
            pred_labels = list(kmeans.labels_)

            perf = computeEvaluationMeasures(trueL, pred_labels)
            runDsResults.append([perf[m] for m in MEASURES])

        runRow = [runSeed]
        runRow.extend(list(np.mean(runDsResults, axis=0)))
        dsresults.append( runRow ) # average

    dsresults.append( ['Average'] + list(np.mean(dsresults, axis=0)[1:]) )
    print(tabulate(dsresults, headers=header, tablefmt='github', floatfmt='.6f', showindex=True))

Setting the path variable:

In [17]:
path = '../stage1_results'

**Build the SRS matrix from a bipartite graph adjacency matrix**

In [ ]:
%%time
embed_and_consensus('3sources', embDim=4, kernelFn='cosine', precomputed_sim=False)

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.383893 | 0.688757 | 0.148218 | 0.149112 | 0.542962 | 0.181696 | 0.149112 | 0.439791 |
|  1 | 102758375 | 0.382750 | 0.693491 | 0.148291 | 0.153846 | 0.543689 | 0.172618 | 0.153846 | 0.440654 |
|  2 | 102758375 | 0.383049 | 0.689349 | 0.214268 | 0.200592 | 0.543564 | 0.262470 | 0.200592 | 0.440430 |
|  3 | 102758375 | 0.382994 | 0.691124 | 0.101892 | 0.107692 | 0.543386 | 0.131759 | 0.107692 | 0.440941 |
|  4 | 102758375 | 0.383061 | 0.692899 | 0.088152 | 0.086391 | 0.543281 | 0.111728 | 0.086391 | 0.440396 |
|  5 | 102758375 | 0.384956 | 0.689941 | 0.141737 | 0.130769 | 0.541923 | 0.179017 | 0.130769 | 0.438402 |
|  6 | 102758375 | 0.330726 | 0.770414 | 0.187258 | 0.188757 | 0.624675 | 0.200181 | 0.188757 | 0.565921 |
|  7 | 102758375 | 0.384825 | 0.692308 | 0.055165 | 0.067456 | 0.541342 | 0.060261 | 0.067456 | 0.438010 |
|  8 | Average   | 0.377032 | 0.701036 | 0.135623 | 0.135577 | 0.553103 | 0.162466 | 0.135577 | 0.455568 |

Wall time: 2.52 s

In [19]:
%%time
embed_and_consensus('BBC-seg2', embDim=4, kernelFn='cosine', precomputed_sim=True)

run 0/10:   0%|          | 0/10 [00:00<?, ?it/s]c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

run 1/10:   0%|          | 0/10 [00:00<?, ?it/s]c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divisio

Opening soft-clustering matrix from ../stage1_results/3sources/simmatrix_DS_Jan042021.015229.out
Storing similarity matrix in sparse format to ../stage1_results/3sources/dotmatrix_DS_Jan042021.015229_csr.npy
../stage1_results/3sources/dotmatrix_DS_Jan042021.015229_csr.npy



run 5/10:  40%|████      | 4/10 [00:00<00:00, 36.04it/s]c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

run 6/10:  40%|████      | 4/10 [00:00<00:00, 36.04it/s]c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. 

Opening soft-clustering matrix from ../stage1_results/3sources/simmatrix_DS_Jan042021.015230.out
Storing similarity matrix in sparse format to ../stage1_results/3sources/dotmatrix_DS_Jan042021.015230_csr.npy
../stage1_results/3sources/dotmatrix_DS_Jan042021.015230_csr.npy



run 7/10:  50%|█████     | 5/10 [00:00<00:00, 37.88it/s]c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

run 8/10:  50%|█████     | 5/10 [00:00<00:00, 37.88it/s]c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. 

...

Opening soft-clustering matrix from ../stage1_results/3sources/simmatrix_DS_Jan042021.015235.out
Storing similarity matrix in sparse format to ../stage1_results/3sources/dotmatrix_DS_Jan042021.015235_csr.npy
../stage1_results/3sources/dotmatrix_DS_Jan042021.015235_csr.npy


c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

run 7/10:  50%|█████     | 5/10 [00:00<00:00, 43.86it/s]c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_p

Opening soft-clustering matrix from ../stage1_results/3sources/simmatrix_DS_Jan042021.015236.out
Storing similarity matrix in sparse format to ../stage1_results/3sources/dotmatrix_DS_Jan042021.015236_csr.npy
../stage1_results/3sources/dotmatrix_DS_Jan042021.015236_csr.npy


c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

run 5/10:  40%|████      | 4/10 [00:00<00:00, 27.39it/s]c:\users\juanz\.virtualenvs\multiview-clustering-n6vzmfus\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.383893 | 0.688757 | 0.148218 | 0.149112 | 0.542962 | 0.181696 | 0.149112 | 0.439791 |
|  1 | 102758375 | 0.382750 | 0.693491 | 0.148291 | 0.153846 | 0.543689 | 0.172618 | 0.153846 | 0.440654 |
|  2 | 102758375 | 0.383049 | 0.689349 | 0.214268 | 0.200592 | 0.543564 | 0.262470 | 0.200592 | 0.440430 |
|  3 | 102758375 | 0.382994 | 0.691124 | 0.101892 | 0.107692 | 0.543386 | 0.131759 | 0.107692 | 0.440941 |
|  4 | 102758375 | 0.383061 | 0.692899 | 0.088152 | 0.086391 | 0.543281 | 0.111728 | 0.086391 | 0.440396 |
|  5 | 102758375 | 0.384956 | 0.689941 | 0.141737 | 0.130769 | 0.541923 | 0.179017 | 0.130769 | 0.438402 |
|  6 | 102758375 | 0.330726 | 0.770414 | 0.187258 | 0.188757 | 0.624675 | 0.200181 | 0.188757 | 0.565921 |
|  7 | 102758375 | 0.384825 | 0.69230

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.383893 | 0.688757 | 0.148218 | 0.149112 | 0.542962 | 0.181696 | 0.149112 | 0.439791 |
|  1 | 102758375 | 0.382750 | 0.693491 | 0.148291 | 0.153846 | 0.543689 | 0.172618 | 0.153846 | 0.440654 |
|  2 | 102758375 | 0.383049 | 0.689349 | 0.214268 | 0.200592 | 0.543564 | 0.262470 | 0.200592 | 0.440430 |
|  3 | 102758375 | 0.382994 | 0.691124 | 0.101892 | 0.107692 | 0.543386 | 0.131759 | 0.107692 | 0.440941 |
|  4 | 102758375 | 0.383061 | 0.692899 | 0.088152 | 0.086391 | 0.543281 | 0.111728 | 0.086391 | 0.440396 |
|  5 | 102758375 | 0.384956 | 0.689941 | 0.141737 | 0.130769 | 0.541923 | 0.179017 | 0.130769 | 0.438402 |
|  6 | 102758375 | 0.330726 | 0.770414 | 0.187258 | 0.188757 | 0.624675 | 0.200181 | 0.188757 | 0.565921 |
|  7 | 102758375 | 0.384825 | 0.692308 | 0.055165 | 0.067456 | 0.541342 | 0.060261 | 0.067456 | 0.438010 |
|  8 | Average   | 0.377032 | 0.701036 | 0.135623 | 0.135577 | 0.553103 | 0.162466 | 0.135577 | 0.455568 |

Wall time: 2.52 s

In [7]:
%%time
embed_and_consensus('BBC-seg2', embDim=4, kernelFn='cosine', precomputed_sim=True)

OSError: ../stage1_results/BBC-seg2/labels.true not found.

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.149259 | 0.951044 | 0.247001 | 0.246521 | 0.851238 | 0.248031 | 0.246521 | 0.881266 |
|  1 | 102758375 | 0.150560 | 0.950497 | 0.209846 | 0.212127 | 0.849929 | 0.207888 | 0.212127 | 0.880068 |
|  2 | 102758375 | 0.152221 | 0.949801 | 0.128924 | 0.129622 | 0.848261 | 0.128573 | 0.129622 | 0.878522 |
|  3 | 102758375 | 0.149619 | 0.950895 | 0.168596 | 0.170278 | 0.850880 | 0.167175 | 0.170278 | 0.880918 |
|  4 | 102758375 | 0.149259 | 0.951044 | 0.196355 | 0.196769 | 0.851238 | 0.196322 | 0.196769 | 0.881266 |
|  5 | 102758375 | 0.152221 | 0.949801 | 0.239271 | 0.240606 | 0.848261 | 0.238347 | 0.240606 | 0.878522 |
|  6 | 102758375 | 0.150920 | 0.950348 | 0.262248 | 0.262425 | 0.849570 | 0.262516 | 0.262425 | 0.879720 |
|  7 | 102758375 | 0.152221 | 0.949801 | 0.265026 | 0.266849 | 0.848261 | 0.263596 | 0.266849 | 0.878522 |
|  8 | 102758375 | 0.152221 | 0.949801 | 0.161928 | 0.162376 | 0.848261 | 0.161830 | 0.162376 | 0.878522 |
|  9 | 102758375 | 0.149259 | 0.951044 | 0.101166 | 0.101143 | 0.851238 | 0.101497 | 0.101143 | 0.881266 |
| 10 | Average   | 0.150776 | 0.950408 | 0.198036 | 0.198872 | 0.849714 | 0.197578 | 0.198872 | 0.879860 |

In [8]:
%%time
embed_and_consensus('BBC-seg3', embDim=4, kernelFn='cosine', precomputed_sim=False)

OSError: ../stage1_results/BBC-seg3/labels.true not found.

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.147757 | 0.951893 | 0.420682 | 0.420110 | 0.850715 | 0.422040 | 0.420110 | 0.881513 |
|  1 | 102758375 | 0.147565 | 0.951972 | 0.314141 | 0.314511 | 0.850922 | 0.314697 | 0.314511 | 0.881687 |
|  2 | 102758375 | 0.147757 | 0.951893 | 0.152430 | 0.155205 | 0.850715 | 0.150422 | 0.155205 | 0.881513 |
|  3 | 102758375 | 0.147757 | 0.951893 | 0.237988 | 0.236041 | 0.850715 | 0.240757 | 0.236041 | 0.881513 |
|  4 | 102758375 | 0.147757 | 0.951893 | 0.179359 | 0.179259 | 0.850715 | 0.180248 | 0.179259 | 0.881513 |
|  5 | 102758375 | 0.147757 | 0.951893 | 0.244366 | 0.243375 | 0.850715 | 0.246283 | 0.243375 | 0.881513 |
|  6 | 102758375 | 0.147757 | 0.951893 | 0.190005 | 0.192823 | 0.850715 | 0.187894 | 0.192823 | 0.881513 |
|  7 | 102758375 | 0.147757 | 0.951893 | 0.195349 | 0.197161 | 0.850715 | 0.194249 | 0.197161 | 0.881513 |
|  8 | 102758375 | 0.147757 | 0.951893 | 0.200190 | 0.199685 | 0.850715 | 0.201537 | 0.199685 | 0.881513 |
|  9 | 102758375 | 0.147757 | 0.951893 | 0.235219 | 0.235095 | 0.850715 | 0.236249 | 0.235095 | 0.881513 |
| 10 | Average   | 0.147738 | 0.951901 | 0.236973 | 0.237326 | 0.850735 | 0.237437 | 0.237326 | 0.881530 |

In [9]:
%%time
embed_and_consensus('Reuters', embDim=4, kernelFn='cosine', precomputed_sim=False)

OSError: ../stage1_results/Reuters/labels.true not found.

In [10]:
%%time
perform_evaluate_runs('BBC-seg4', embDim=4, kernelFn='cosine')

NameError: name 'perform_evaluate_runs' is not defined

dim:4, cosine

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.183388 | 0.932847 | 0.305188 | 0.304526 | 0.807615 | 0.309141 | 0.304526 | 0.835237 |
|  1 | 102758375 | 0.181714 | 0.933285 | 0.098665 | 0.100000 | 0.809284 | 0.101374 | 0.100000 | 0.836075 |
|  2 | 102758375 | 0.183388 | 0.932847 | 0.120351 | 0.122336 | 0.807615 | 0.120372 | 0.122336 | 0.835237 |
|  3 | 102758375 | 0.184038 | 0.932555 | 0.139719 | 0.142774 | 0.806961 | 0.141232 | 0.142774 | 0.834496 |
|  4 | 102758375 | 0.182830 | 0.932993 | 0.116874 | 0.117664 | 0.808172 | 0.120214 | 0.117664 | 0.835517 |
|  5 | 102758375 | 0.183388 | 0.932847 | 0.161820 | 0.164088 | 0.807615 | 0.162908 | 0.164088 | 0.835237 |
|  6 | 102758375 | 0.182830 | 0.932993 | 0.090352 | 0.090219 | 0.808172 | 0.094477 | 0.090219 | 0.835517 |
|  7 | 102758375 | 0.182830 | 0.932993 | 0.138518 | 0.140146 | 0.808172 | 0.140495 | 0.140146 | 0.835517 |
|  8 | 102758375 | 0.181985 | 0.933139 | 0.171865 | 0.174891 | 0.808953 | 0.173101 | 0.174891 | 0.835747 |
|  9 | 102758375 | 0.184038 | 0.932555 | 0.208822 | 0.211095 | 0.806961 | 0.210462 | 0.211095 | 0.834496 |
| 10 | Average   | 0.183043 | 0.932905 | 0.155217 | 0.156774 | 0.807952 | 0.157378 | 0.156774 | 0.835308 |
CPU times: user 8.88 s, sys: 4.01 s, total: 12.9 s
Wall time: 2.95 s



|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.268472 | 0.894161 | 0.149339 | 0.155328 | 0.721357 | 0.148146 | 0.155328 | 0.755605 |
|  1 | 102758375 | 0.268042 | 0.894307 | 0.094929 | 0.097664 | 0.721754 | 0.098474 | 0.097664 | 0.755752 |
|  2 | 102758375 | 0.269831 | 0.893431 | 0.118097 | 0.124380 | 0.720049 | 0.117356 | 0.124380 | 0.754138 |
|  3 | 102758375 | 0.269305 | 0.893723 | 0.210728 | 0.215182 | 0.720558 | 0.211137 | 0.215182 | 0.754575 |
|  4 | 102758375 | 0.266446 | 0.895182 | 0.153666 | 0.154599 | 0.723294 | 0.158414 | 0.154599 | 0.757712 |
|  5 | 102758375 | 0.267779 | 0.894453 | 0.135532 | 0.139416 | 0.722008 | 0.136482 | 0.139416 | 0.755971 |
|  6 | 102758375 | 0.268304 | 0.894307 | 0.133509 | 0.138832 | 0.721531 | 0.132912 | 0.138832 | 0.755996 |
|  7 | 102758375 | 0.267542 | 0.894599 | 0.146864 | 0.150949 | 0.722240 | 0.148295 | 0.150949 | 0.756463 |
|  8 | 102758375 | 0.272487 | 0.890219 | 0.300385 | 0.306861 | 0.716485 | 0.301102 | 0.306861 | 0.749963 |
|  9 | 102758375 | 0.268972 | 0.893869 | 0.039151 | 0.043942 | 0.720872 | 0.040679 | 0.043942 | 0.754894 |
| 10 | Average   | 0.268718 | 0.893825 | 0.148220 | 0.152715 | 0.721015 | 0.149300 | 0.152715 | 0.755107 |

CPU times: user 4min 56s, sys: 2.46 s, total: 4min 58s
Wall time: 4min 46s

In [11]:
%%time
perform_evaluate_runs('Caltech', embDim=8, kernelFn='cosine')

NameError: name 'perform_evaluate_runs' is not defined

dim:8, cosine

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.428383 | 0.659640 | 0.043620 | 0.044007 | 0.436066 | 0.073938 | 0.044007 | 0.303957 |
|  1 | 102758375 | 0.427989 | 0.662825 | 0.065009 | 0.058256 | 0.436709 | 0.097686 | 0.058256 | 0.318946 |
|  2 | 102758375 | 0.418265 | 0.652850 | 0.113960 | 0.106873 | 0.450076 | 0.199417 | 0.106873 | 0.346506 |
|  3 | 102758375 | 0.419574 | 0.652096 | 0.068019 | 0.066345 | 0.444782 | 0.104811 | 0.066345 | 0.336888 |
|  4 | 102758375 | 0.429605 | 0.658466 | 0.051974 | 0.055532 | 0.436864 | 0.076387 | 0.055532 | 0.318954 |
|  5 | 102758375 | 0.428818 | 0.664878 | 0.062528 | 0.059137 | 0.439885 | 0.154630 | 0.059137 | 0.314035 |
|  6 | 102758375 | 0.425140 | 0.645013 | 0.048215 | 0.044593 | 0.439948 | 0.109057 | 0.044593 | 0.335990 |
|  7 | 102758375 | 0.428611 | 0.663118 | 0.057425 | 0.056370 | 0.439868 | 0.097537 | 0.056370 | 0.314683 |
|  8 | 102758375 | 0.437024 | 0.656035 | 0.049690 | 0.044677 | 0.430078 | 0.095058 | 0.044677 | 0.333822 |
|  9 | 102758375 | 0.415802 | 0.653353 | 0.033990 | 0.034996 | 0.449886 | 0.052390 | 0.034996 | 0.351205 |
| 10 | Average   | 0.425921 | 0.656827 | 0.059443 | 0.057079 | 0.440416 | 0.106091 | 0.057079 | 0.327498 |
CPU times: user 31.9 s, sys: 9.97 s, total: 41.9 s
Wall time: 11 s


dim:5 , cosine

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.429888 | 0.648910 | 0.052559 | 0.052096 | 0.425421 | 0.078130 | 0.052096 | 0.322740 |
|  1 | 102758375 | 0.430596 | 0.649581 | 0.050770 | 0.046773 | 0.423121 | 0.094186 | 0.046773 | 0.316401 |
|  2 | 102758375 | 0.405382 | 0.659514 | 0.094435 | 0.085247 | 0.450282 | 0.159397 | 0.085247 | 0.354355 |
|  3 | 102758375 | 0.407962 | 0.655448 | 0.073609 | 0.064920 | 0.448392 | 0.125081 | 0.064920 | 0.360443 |
|  4 | 102758375 | 0.430999 | 0.648701 | 0.023385 | 0.027494 | 0.420953 | 0.038343 | 0.027494 | 0.309670 |
|  5 | 102758375 | 0.431033 | 0.650838 | 0.057918 | 0.048868 | 0.422462 | 0.109296 | 0.048868 | 0.314567 |
|  6 | 102758375 | 0.409791 | 0.655029 | 0.064198 | 0.054862 | 0.445235 | 0.138087 | 0.054862 | 0.348076 |
|  7 | 102758375 | 0.429922 | 0.648868 | 0.058080 | 0.049162 | 0.425611 | 0.120052 | 0.049162 | 0.328405 |
|  8 | 102758375 | 0.422801 | 0.661065 | 0.071160 | 0.065423 | 0.433804 | 0.118065 | 0.065423 | 0.350798 |
|  9 | 102758375 | 0.407643 | 0.656329 | 0.065358 | 0.062531 | 0.448731 | 0.142431 | 0.062531 | 0.349878 |
| 10 | Average   | 0.420602 | 0.653428 | 0.061147 | 0.055738 | 0.434401 | 0.112307 | 0.055738 | 0.335533 |
CPU times: user 31.4 s, sys: 9.99 s, total: 41.4 s
Wall time: 10.9 s

dim:3, cosine

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.429080 | 0.649246 | 0.030382 | 0.031140 | 0.425903 | 0.045765 | 0.031140 | 0.350819 |
|  1 | 102758375 | 0.430828 | 0.649329 | 0.034217 | 0.034367 | 0.423641 | 0.051573 | 0.034367 | 0.345043 |
|  2 | 102758375 | 0.414870 | 0.643713 | 0.093608 | 0.080134 | 0.433554 | 0.189920 | 0.080134 | 0.312642 |
|  3 | 102758375 | 0.415314 | 0.643588 | 0.055367 | 0.047946 | 0.432987 | 0.114854 | 0.047946 | 0.312657 |
|  4 | 102758375 | 0.427223 | 0.650168 | 0.074751 | 0.065130 | 0.427829 | 0.117880 | 0.065130 | 0.352501 |
|  5 | 102758375 | 0.429397 | 0.648156 | 0.040202 | 0.037427 | 0.422792 | 0.073292 | 0.037427 | 0.339596 |
|  6 | 102758375 | 0.417945 | 0.643588 | 0.058441 | 0.053646 | 0.428766 | 0.116574 | 0.053646 | 0.299925 |
|  7 | 102758375 | 0.429430 | 0.650754 | 0.049732 | 0.045851 | 0.427227 | 0.093415 | 0.045851 | 0.357674 |
|  8 | 102758375 | 0.428885 | 0.648659 | 0.057716 | 0.059262 | 0.420952 | 0.088058 | 0.059262 | 0.325333 |
|  9 | 102758375 | 0.414440 | 0.643965 | 0.068801 | 0.054694 | 0.432413 | 0.199044 | 0.054694 | 0.310104 |
| 10 | Average   | 0.423741 | 0.647117 | 0.056322 | 0.050960 | 0.427606 | 0.109038 | 0.050960 | 0.330629 |
CPU times: user 31.9 s, sys: 9.51 s, total: 41.4 s
Wall time: 11.2 s


dim : 3

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.445440 | 0.643336 | 0.026973 | 0.027033 | 0.414287 | 0.055858 | 0.027033 | 0.342926 |
|  1 | 102758375 | 0.444238 | 0.643336 | 0.045111 | 0.041408 | 0.416247 | 0.070447 | 0.041408 | 0.348293 |
|  2 | 102758375 | 0.424884 | 0.648198 | 0.077482 | 0.071961 | 0.426085 | 0.128635 | 0.071961 | 0.304730 |
|  3 | 102758375 | 0.423435 | 0.648198 | 0.077158 | 0.065339 | 0.426261 | 0.186978 | 0.065339 | 0.301516 |
|  4 | 102758375 | 0.443114 | 0.643168 | 0.046479 | 0.041869 | 0.415925 | 0.073534 | 0.041869 | 0.346467 |
|  5 | 102758375 | 0.441940 | 0.644216 | 0.027960 | 0.033445 | 0.415597 | 0.042074 | 0.033445 | 0.339148 |
|  6 | 102758375 | 0.422699 | 0.645306 | 0.045728 | 0.044216 | 0.426639 | 0.077841 | 0.044216 | 0.304276 |
|  7 | 102758375 | 0.443025 | 0.644258 | 0.057770 | 0.052054 | 0.419893 | 0.104900 | 0.052054 | 0.376782 |
|  8 | 102758375 | 0.444823 | 0.644593 | 0.035458 | 0.036421 | 0.411878 | 0.068847 | 0.036421 | 0.349812 |
|  9 | 102758375 | 0.422160 | 0.648617 | 0.055328 | 0.041953 | 0.428452 | 0.199043 | 0.041953 | 0.308407 |
| 10 | Average   | 0.435576 | 0.645323 | 0.049545 | 0.045570 | 0.420126 | 0.100816 | 0.045570 | 0.332236 |

CPU times: user 1min 46s, sys: 57.2 s, total: 2min 44s
Wall time: 28.2 s

In [12]:
%%time
perform_evaluate_runs('handwritten', embDim=6, kernelFn='cosine') # no important difference with the default thresold

NameError: name 'perform_evaluate_runs' is not defined

dim:6, cosine 

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.281818 | 0.761050 | 0.095272 | 0.099400 | 0.730629 | 0.101884 | 0.099400 | 0.627191 |
|  1 | 102758375 | 0.276767 | 0.770700 | 0.069029 | 0.077700 | 0.734134 | 0.066716 | 0.077700 | 0.637387 |
|  2 | 102758375 | 0.277524 | 0.764200 | 0.138228 | 0.145250 | 0.733049 | 0.142899 | 0.145250 | 0.636140 |
|  3 | 102758375 | 0.278512 | 0.770100 | 0.119809 | 0.123950 | 0.732739 | 0.125409 | 0.123950 | 0.632471 |
|  4 | 102758375 | 0.278240 | 0.764150 | 0.095165 | 0.102700 | 0.732373 | 0.098505 | 0.102700 | 0.630341 |
|  5 | 102758375 | 0.284441 | 0.763200 | 0.074636 | 0.077600 | 0.729158 | 0.077518 | 0.077600 | 0.619431 |
|  6 | 102758375 | 0.275399 | 0.774150 | 0.118619 | 0.124000 | 0.734071 | 0.118853 | 0.124000 | 0.640756 |
|  7 | 102758375 | 0.302031 | 0.742150 | 0.116881 | 0.123550 | 0.713664 | 0.121963 | 0.123550 | 0.605184 |
|  8 | 102758375 | 0.277821 | 0.765100 | 0.100481 | 0.112100 | 0.732227 | 0.098190 | 0.112100 | 0.634336 |
|  9 | 102758375 | 0.279416 | 0.768800 | 0.075536 | 0.075800 | 0.732158 | 0.077696 | 0.075800 | 0.627847 |
| 10 | Average   | 0.281197 | 0.764360 | 0.100366 | 0.106205 | 0.730420 | 0.102963 | 0.106205 | 0.629108 |
CPU times: user 21.4 s, sys: 7.06 s, total: 28.5 s
Wall time: 7.18 s

dim : 6

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.290102 | 0.763400 | 0.121379 | 0.121450 | 0.716668 | 0.125602 | 0.121450 | 0.634751 |
|  1 | 102758375 | 0.289881 | 0.763200 | 0.073071 | 0.075850 | 0.716801 | 0.073154 | 0.075850 | 0.630876 |
|  2 | 102758375 | 0.290499 | 0.761100 | 0.132062 | 0.133500 | 0.715696 | 0.134442 | 0.133500 | 0.634909 |
|  3 | 102758375 | 0.288695 | 0.764150 | 0.095413 | 0.098150 | 0.717387 | 0.095284 | 0.098150 | 0.634717 |
|  4 | 102758375 | 0.293103 | 0.758750 | 0.124142 | 0.126900 | 0.715118 | 0.127762 | 0.126900 | 0.627984 |
|  5 | 102758375 | 0.289417 | 0.762350 | 0.100434 | 0.099700 | 0.717925 | 0.106166 | 0.099700 | 0.630034 |
|  6 | 102758375 | 0.289903 | 0.763300 | 0.145624 | 0.148700 | 0.716421 | 0.149754 | 0.148700 | 0.632362 |
|  7 | 102758375 | 0.337331 | 0.688300 | 0.098746 | 0.100600 | 0.673209 | 0.105153 | 0.100600 | 0.583970 |
|  8 | 102758375 | 0.290753 | 0.762650 | 0.080585 | 0.080400 | 0.717223 | 0.086493 | 0.080400 | 0.628180 |
|  9 | 102758375 | 0.290330 | 0.762950 | 0.071886 | 0.069750 | 0.716495 | 0.078104 | 0.069750 | 0.634831 |
| 10 | Average   | 0.295001 | 0.755015 | 0.104334 | 0.105500 | 0.712294 | 0.108191 | 0.105500 | 0.627261 |

CPU times: user 56.9 s, sys: 31.1 s, total: 1min 28s
Wall time: 15 s

In [13]:
perform_evaluate_runs('NusWide', embDim=5, kernelFn='cosine') # no important difference with the default thresold

NameError: name 'perform_evaluate_runs' is not defined

---

In [ ]:
dsDirPath = 'BBC-seg4'
simFname = "{0}/{1}/{2}".format(path, dsDirPath, 'simmatrix_DS_Nov262020.182009_csr.npy')
trueLblsFname = "{0}/{1}/{2}".format(path, dsDirPath, 'labels.true')
trueL = getLabelsFromFile(trueLblsFname)
D = _loadSparseCSRFromDisk(simFname)
D

In [ ]:
simClus = getSimClus(D)
simClus.shape

In [ ]:
%%time
aSRS = compute_aSRS(D)

Obtaining the graph embedding

In [ ]:
from spectral_embedding import *

spectralEmb = SpectralEmbedding(embedding_dimension = 30, node_weights = 'degree')
spectralEmb.fit(aSRS)

embedding = spectralEmb.embedding_
embedding.shape

from sklearn.cluster import KMeans, MiniBatchKMeans

n_clusters = 5

kmeans = MiniBatchKMeans(n_clusters)
kmeans.fit(embedding)
pred_labels = list(kmeans.labels_)
np.unique(pred_labels, return_counts=True)

from clustering_evaluation import computeEvaluationMeasures, MEASURES

perf = computeEvaluationMeasures(trueL, pred_labels) 
perf

In [ ]:
from sklearn.manifold import SpectralEmbedding, spectral_embedding, LocallyLinearEmbedding
from clustering_evaluation import computeEvaluationMeasures, MEASURES
from sklearn.cluster import MiniBatchKMeans

In [ ]:
%%time
# builds the embedded vectors
embedding = SpectralEmbedding(n_components=20, affinity='precomputed', n_jobs=4)
D_transformed = embedding.fit_transform(csr_matrix(aSRS))
D_transformed.shape

In [ ]:
# clustering the embedded vectors
n_clusters = 5

kmeans = MiniBatchKMeans(n_clusters)
kmeans.fit(D_transformed)
pred_labels = list(kmeans.labels_)
np.unique(pred_labels, return_counts=True)

# evaluates
perf = computeEvaluationMeasures(trueL, pred_labels) 
perf

In [ ]:
D_transformed = spectral_embedding(aSRS, n_components=4)

In [ ]:
# clustering the embedded vectors
n_clusters = 5
results = []
for i in range(10):
    kmeans = MiniBatchKMeans(n_clusters)
    kmeans.fit(D_transformed)
    pred_labels = list(kmeans.labels_)
    np.unique(pred_labels, return_counts=True)

    # evaluates
    perf = computeEvaluationMeasures(trueL, pred_labels)
    results.append([perf[m] for m in MEASURES])    

results.append(list(np.mean(results, axis=0)))

print(tabulate(results, headers=MEASURES, tablefmt='github'))

In [ ]:
Sp = np.abs(np.random.randn(4,3))
Sp

In [ ]:
from sklearn.metrics.pairwise import pairwise_kernels

pairwise_kernels(Sp, metric='cosine')

In [ ]:
pairwise_kernels(Sp, metric='rbf')

In [ ]:
pairwise_kernels(Sp, metric='laplacian')

In [ ]:
Sp.dot(Sp.T)